In [1]:
import json
import pandas as pd

In [123]:
def read_from_json(file_name):
    with open(f'{file_name}') as f:
        data = json.load(f)
    return data

In [3]:
recipes_ingredient = read_from_json('../dataset/recipes_ingredient.json')

### Check wheather the unit of ingredents are uniformed or not

In [142]:
# dict to get all unit for each ingredent
ingredients_unit = {}

# dict to get recipe id and its ingredent with ingredent name
recipe_with_ingredient = {}
for r in recipes_ingredient:
    r_id = r['recipe_id']
    recipe_with_ingredient[r_id] = []
    cur = r['ingredients']['ingredient']
    if isinstance(cur, list):
        for food in cur:
            k = (food['food_id'], food['food_name'])
            recipe_with_ingredient[r_id].append(k)
            m = food['measurement_description']
            if k not in ingredients_unit:
                ingredients_unit[k] = []
            if m not in ingredients_unit[k]:
                ingredients_unit[k].append(m)
    else:
        k = (cur['food_id'], cur['food_name'])
        recipe_with_ingredient[r_id].append(k)
        m = cur['measurement_description']
        if k not in ingredients_unit:
            ingredients_unit[k] = []
        if m not in ingredients_unit[k]:
            ingredients_unit[k].append(m)


In [143]:
# get all food id
all_food_id = set()
for r in recipes_ingredient:
    cur = r['ingredients']['ingredient']
    if isinstance(cur, list):
        for food in cur:
            all_food_id.add(food['food_id'])
    else:
         all_food_id.add(cur['food_id'])

In [144]:
# make sure the id is consistant with name
len(all_food_id) == len(ingredients_unit)

True

In [145]:
# show the unit for each ingredent
list(ingredients_unit.items())[:10]

[(('6629', 'Yeast'), ['tablespoon, dry', 'g']),
 (('39536', 'Honey'), ['tbsp', 'cup', 'oz']),
 (('3092', 'Egg'), ['large', 'extra large', 'jumbo', 'medium', 'small']),
 (('7048', 'Vegetable Oil'), ['tablespoon', 'cup']),
 (('263348', 'Apple Cider Vinegar'), ['serving']),
 (('12037965', 'Gluten Free 1 to 1 Baking Flour'), ['serving']),
 (('36492', 'Russet Potatoes (Flesh and Skin)'),
  ['Potato medium (2-1/4" to 3-1/4" dia)',
   'lb',
   'cup, diced',
   'potato small (1-3/4" to 2-1/4" dia)',
   'Potato large (3" to 4-1/4" dia)']),
 (('21628901', 'Himalayan Pink Salt'), ['serving']),
 (('33882', 'Garlic Powder'), ['tsp', 'tbsp', 'g']),
 (('33890', 'Paprika'), ['tsp', 'tbsp'])]

***
##### As shown above the unit of each ingredent is not consistant

In [146]:
list(recipe_with_ingredient.items())[:2]

[('45430986',
  [('6629', 'Yeast'),
   ('39536', 'Honey'),
   ('3092', 'Egg'),
   ('7048', 'Vegetable Oil'),
   ('263348', 'Apple Cider Vinegar'),
   ('12037965', 'Gluten Free 1 to 1 Baking Flour')]),
 ('42818205',
  [('36492', 'Russet Potatoes (Flesh and Skin)'),
   ('21628901', 'Himalayan Pink Salt'),
   ('33882', 'Garlic Powder'),
   ('33890', 'Paprika'),
   ('34304', 'Avocado Vegetable Oil')])]

### Use 1/0 to indicate wheather the ingredent is contained in a recipe or not

In [147]:
all_food_id_dict = {}
count = 0
for i in all_food_id:
    all_food_id_dict[i] = count
    count += 1

In [148]:
recipe_dataset = {}
for r_id, foods in recipe_with_ingredient.items():
    vector = [0] * len(all_food_id)
    for f in foods:
        food_id = f[0]
        position = all_food_id_dict[food_id]
        vector[position] = 1
    recipe_dataset[r_id] = vector

In [149]:
recipe_df = pd.DataFrame.from_dict(recipe_dataset, orient='index', columns = all_food_id_dict.keys())

In [150]:
recipe_df.head()

33919  39699  61271  3419  15845548  75372  570035  1071192  41011  \
45430986      0      0      0     0         0      0       0        0      0   
42818205      0      0      0     0         0      0       0        0      0   
42384638      0      0      0     0         0      0       0        0      0   
42288128      0      0      0     0         0      0       0        0      0   
8629075       0      0      0     0         0      0       0        0      0   

          39128  95051  370626  49651  2704756  47613  40696  52919  91273  \
45430986      0      0       0      0        0      0      0      0      0   
42818205      0      0       0      0        0      0      0      0      0   
42384638      0      0       0      0        0      0      0      0      0   
42288128      0      0       0      0        0      0      0      0      0   
8629075       0      0       0      0        0      0      0      0      0   

          74198  7929133  36351  2079878  33833  27452  6513867  39765  37784  \
45430986      0        0      0        0      0      0        0      0      0   
42818205      0        0      0        0      0      0        0      0      0   
42384638      0        0      0        0      0      0        0      0      0   
42288128      0        0      0        0      0      0        0      0      0   
8629075       0        0      0        0      0      0        0      0      0   

          39064  1052  39814  3060772  48246  399077  516013  229053  406742  \
45430986      0     0      0        0      0       0       0       0       0   
42818205      0     0      0        0      0       0       0       0       0   
42384638      0     0      0        0      0       0       0       0       0   
42288128      0     0      0        0      0       0       0       0       0   
8629075       0     0      0        0      0       0       0       0       0   

          49239  2727224  36638  39730  47786  6138  15830107  31242  6211629  \
45430986      0        0      0      0      0     0         0      0        0   
42818205      0        0      0      0      0     0         0      0        0   
42384638      0        0      0      0      0     0         0      0        0   
42288128      0        0      0      0      0     1         0      0        0   
8629075       0        0      0      0      0     0         0      0        0   

          492000  35953  66026  66054  121036  33793  37031  57542  27304  \
45430986       0      0      0      0       0      0      0      0      0   
42818205       0      0      0      0       0      0      0      0      0   
42384638       0      0      0      0       0      0      0      0      0   
42288128       0      0      0      0       0      0      0      0      0   
8629075        0      0      0      0       0      0      0      0      0   

          253129  36771  2305991  5104775  23763376  39726  36721  274065  \
45430986       0      0        0        0         0      0      0       0   
42818205       0      0        0        0         0      0      0       0   
42384638       0      0        0        0         0      0      0       0   
42288128       0      0        0        0         0      0      0       0   
8629075        0      0        0        0         0      0      0       0   

          6389369  43303  38250  36981  404819  988992  34245  564941  39058  \
45430986        0      0      0      0       0       0      0       0      0   
42818205        0      0      0      0       0       0      0       0      0   
42384638        0      0      0      0       0       0      0       0      0   
42288128        0      0      0      0       0       0      0       0      0   
8629075         0      0      0      0       0       0      0       0      0   

          55103  37966  7150085  906667  69602  1887  33686  39534  35423  \
45430986      0      0        0       0      0     0      0      0      0   
42818205      0      0        0       0     

### Food_id and it's name

In [154]:
food_df = pd.DataFrame(ingredients_unit.keys(),columns = ['food_id', 'food_name'])

In [155]:
food_df.head()

,food_id,food_name
0,6629,Yeast
1,39536,Honey
2,3092,Egg
3,7048,Vegetable Oil
4,263348,Apple Cider Vinegar


In [157]:
#food_df.to_csv('food.csv',index=False)
recipe_df.to_csv('recipe_with_ingredient.csv')